In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# Load the dataset
dataset_path = '/Users/chiragprabhakar/Documents/data_analytics_A1/final_assgnment/dataset - 2020-09-24.csv'
data = pd.read_csv(dataset_path)



FileNotFoundError: [Errno 2] No such file or directory: '/Users/chiragprabhakar/Documents/data_analytics_A1/final_assgnment/dataset - 2020-09-24.csv'

In [ ]:
# Display dataset structure and the first five rows
print("Data Info:")
data.info()
print("\nFirst 5 Rows:")
data.head()

In [ ]:
#cleaning the data set
# Drop columns with excessive missing values (threshold: <30% non-null)
threshold = 0.3 * len(data)
data = data.dropna(axis=1, thresh=threshold)

#Fill missing numeric values with column mean
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

# Fill missing categorical values with mode
categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    data[col] = data[col].fillna(data[col].mode()[0])

# Preview cleaned dataset
data.info()
data.head()

In [ ]:
# Remove entries which do not have age, jersey number and nationality
data = data[data['Nationality'].notna()]
data = data[data['Age'].notna()]
data = data[data['Jersey Number'].notna()]

# cleaning the percentage sign
data['Cross accuracy %'] = data['Cross accuracy %'].str.replace(r'%', '').astype(float)
data['Shooting accuracy %'] = data['Shooting accuracy %'].str.replace(r'%', '').astype(float)
data['Tackle success %'] = data['Tackle success %'].str.replace(r'%', '').astype(float)

features = data.columns
data_clean = data[features]
data_clean.head()

data_clean_appNonZero = data_clean[data_clean['Appearances'] > 0] #prevents division by zero for players who are yet to make appearances

# take care off the data type for division
# separate cols with dtype objects and cols that may not be divided by appearances

cols = features.drop(['Age', 'Name', 'Appearances', 'Club', 'Nationality', 'Jersey Number', 'Cross accuracy %', 'Position', 'Goals per match',
                      'Passes per match','Tackle success %', 'Shooting accuracy %'])
data_clean_appNonZero.loc[:, cols] = data_clean_appNonZero.loc[:, cols].div(data_clean_appNonZero['Appearances'], axis=0)

# positional classifications on the data as is
goalies = data[data['Position'] == 'Goalkeeper']
defenders = data[data['Position'] == 'Defender']
midfielders = data[data['Position'] == 'Midfielder']
forwards = data[data['Position'] == 'Forward']

# palyers who have made atleast 38 games (a seasons worth games)
# data as is
data_38app = data[data['Appearances'] >=38]
goalies_38app = goalies[goalies['Appearances'] >= 38]
defenders_38app = defenders[defenders['Appearances'] >= 38]
midfilders_38app = midfielders[midfielders['Appearances'] >= 38]
forwards_38app = forwards[forwards['Appearances'] >= 38]

# palyers who have made atleast 38 games (a seasons worth games)
# data normalized
all_players = data_clean_appNonZero[data_clean_appNonZero['Appearances'] >= 38]
goalies_ = data_clean_appNonZero[(data_clean_appNonZero['Position'] == 'Goalkeeper') & (data_clean_appNonZero['Appearances'] >= 38)]
defenders_ = data_clean_appNonZero[(data_clean_appNonZero['Position'] == 'Defender') & (data_clean_appNonZero['Appearances'] >= 38)]
midfielders_ = data_clean_appNonZero[(data_clean_appNonZero['Position'] == 'Midfielder') & (data_clean_appNonZero['Appearances'] >= 38)]
forwards_ = data_clean_appNonZero[(data_clean_appNonZero['Position'] == 'Forward') & (data_clean_appNonZero['Appearances'] >= 38)]

In [ ]:
# Save DataFrame to Excel
data.to_excel("cleaned_EPL_dataset.xlsx", index=False)

In [ ]:
data.head()

In [ ]:
# EDA: Distribution of Goals
plt.figure(figsize=(10, 6))
sns.histplot(data['Goals'], bins=20, kde=True, color='blue')
plt.title('Distribution of Goals Scored')
plt.xlabel('Goals')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# EDA: Correlation heatmap for numerical columns
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns
corr = data[numerical_cols].corr()
plt.figure(figsize=(15, 12))
sns.heatmap(corr, annot=False, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Hypothesis 1: Linear Regression to Predict Goals
print("\nLinear Regression: Predicting Goals")
X = data[['Appearances', 'Shots', 'Passes per match']].dropna()
y = data['Goals'].loc[X.index]

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Predictions
y_pred = lin_reg.predict(X_test)

# Evaluation
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-squared Score:", r2_score(y_test, y_pred))

# Display Coefficients
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': lin_reg.coef_
})
print("\nCoefficients:")
print(coefficients)

In [ ]:
# Hypothesis 2: K-Means Clustering to Group Players
print("\nK-Means Clustering")
kmeans_features = data[['Goals', 'Assists', 'Passes', 'Tackles']].dropna()
scaler = StandardScaler()
kmeans_scaled = scaler.fit_transform(kmeans_features)

# Model Training
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(kmeans_scaled)

# Assign Cluster Labels
data['Cluster'] = None
data.loc[kmeans_features.index, 'Cluster'] = kmeans.labels_

# Visualize Clusters
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=kmeans_features['Goals'],
    y=kmeans_features['Assists'],
    hue=kmeans.labels_,
    palette='viridis',
    legend='full'
)
plt.title('K-Means Clustering: Player Performance')
plt.xlabel('Goals')
plt.ylabel('Assists')
plt.show()

print("Clustering Complete. Assigned clusters added to dataset.")

In [ ]:
# Question1 - Countries most represented in EPL

df = data
fig = px.pie(df,
             values='Appearances',
             names='Nationality',
             title='Countries represented in the EPL by number of appearances',
             )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(title_text='<b> Nr. of appearances per country <b>',
                  title_x=0.5,
                  titlefont=dict(color='black',
                            size=28,
                            family="Courier New, monospace",),
                  width=600,
                  height=600,
                  showlegend=False,
                 )
iplot(fig)

###########################

nationality_counts = data['Nationality'].value_counts().head(10)  # Top 10 countries

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(x=nationality_counts.values, y=nationality_counts.index, palette="viridis")
plt.title('Top 10 Countries Represented in the EPL', fontsize=16)
plt.xlabel('Number of Players', fontsize=12)
plt.ylabel('Nationality', fontsize=12)
plt.show()


In [ ]:
# Question2 - Clubs giving more chances to youngsters and their winning percentage

# Define youngsters as players aged 21 or younger
youngsters = data[data['Age'] <= 21]
youngsters_club = youngsters['Club'].value_counts()

# Calculate winning percentage for each club
data['Winning Percentage'] = (data['Wins'] / data['Appearances']) * 100
data_club_win = data.groupby('Club')['Winning Percentage'].mean()

# Merge youngster count and winning percentage
youngsters_win_data = pd.DataFrame({
    'Youngsters Count': youngsters_club,
    'Average Winning %': data_club_win
}).dropna()

# Plot
fig, ax1 = plt.subplots(figsize=(14, 8))

color = 'tab:blue'
ax1.set_title('Youngsters Count vs Average Winning Percentage by Club', fontsize=16)
ax1.set_xlabel('Clubs', fontsize=12)
ax1.set_ylabel('Youngsters Count', color=color, fontsize=12)
ax1.bar(youngsters_win_data.index, youngsters_win_data['Youngsters Count'], color=color, alpha=0.7)
ax1.tick_params(axis='x', rotation=90)

ax2 = ax1.twinx()  # Instantiate a second y-axis
color = 'tab:green'
ax2.set_ylabel('Average Winning %', color=color, fontsize=12)
ax2.plot(youngsters_win_data.index, youngsters_win_data['Average Winning %'], color=color, marker='o')
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()
plt.show()


In [ ]:
# Question 3 - Does average age on the higher side leads to more wins than the squad with less average side.

# Drop rows with missing age or performance metrics
data = data.dropna(subset=['Age', 'Appearances', 'Wins', 'Goals'])

# Define age groups
def categorize_age(age):
    if age < 20:
        return 'Under 20'
    elif 20 <= age < 25:
        return '20-24'
    elif 25 <= age < 30:
        return '25-29'
    elif 30 <= age < 35:
        return '30-34'
    else:
        return '35 and above'

# Apply age group categorization
data['Age Group'] = data['Age'].apply(categorize_age)

# Calculate average performance metrics for each age group
age_group_stats = data.groupby('Age Group').agg({
    'Appearances': 'mean',
    'Wins': 'mean',
    'Goals': 'mean'
}).reset_index()

# Sort age groups for better visualization
age_group_stats = age_group_stats.sort_values(by='Age Group', key=lambda x: x.map({
    'Under 20': 0,
    '20-24': 1,
    '25-29': 2,
    '30-34': 3,
    '35 and above': 4
}))

# Plot the results
plt.figure(figsize=(10, 6))

# Plot metrics for each age group
x = age_group_stats['Age Group']
plt.bar(x, age_group_stats['Wins'], color='skyblue', label='Average Wins', alpha=0.7)
plt.plot(x, age_group_stats['Goals'], marker='o', color='orange', label='Average Goals', linestyle='-', linewidth=2)
plt.plot(x, age_group_stats['Appearances'], marker='s', color='green', label='Average Appearances', linestyle='--', linewidth=2)

# Add labels, legend, and title
plt.xlabel('Age Group')
plt.ylabel('Performance Metrics')
plt.title('Performance Metrics by Age Group')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Question 4 - Which positions contribute most to goals and assists?" the question

# Group data by position and calculate total goals and assists
position_contributions = data.groupby('Position')[['Goals', 'Assists']].sum().reset_index()

# Sort by goals for better visualization
position_contributions = position_contributions.sort_values('Goals', ascending=False)

# Visualization: Bar plot for goals and assists by position
plt.figure(figsize=(12, 6))
bar_width = 0.4
x = position_contributions['Position']
bar1 = plt.bar(x, position_contributions['Goals'], width=bar_width, label='Goals', color='skyblue')
bar2 = plt.bar(x, position_contributions['Assists'], width=bar_width, label='Assists', color='orange', bottom=position_contributions['Goals'])

# Adding labels
plt.title('Goals and Assists Contribution by Position', fontsize=16)
plt.xlabel('Position', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.legend()
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Visualization: Stacked percentage contribution
position_contributions['Total'] = position_contributions['Goals'] + position_contributions['Assists']
position_contributions['Goal Percentage'] = (position_contributions['Goals'] / position_contributions['Total']) * 100
position_contributions['Assist Percentage'] = (position_contributions['Assists'] / position_contributions['Total']) * 100

plt.figure(figsize=(12, 6))
sns.barplot(data=position_contributions, x='Position', y='Total', palette='viridis')
plt.title('Total Contribution to Goals and Assists by Position', fontsize=16)
plt.xlabel('Position', fontsize=14)
plt.ylabel('Total Goals and Assists', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Question 5 - What are the data we need to look into to find the best goalkeeper.


# Filter only goalkeepers
goalkeepers = data[data['Position'] == 'Goalkeeper']

# Relevant metrics for goalkeepers
goalkeeper_metrics = [
    'Name', 'Club', 'Appearances', 'Clean sheets', 'Goals conceded', 'Saves',
    'Penalties saved', 'Punches', 'High Claims', 'Catches', 'Sweeper clearances',
    'Throw outs', 'Goal Kicks', 'Errors leading to goal'
]

# Subset the data for analysis
goalkeepers_data = goalkeepers[goalkeeper_metrics]

# Drop rows with missing critical data (like Clean sheets, Saves, etc.)
goalkeepers_cleaned = goalkeepers_data.dropna(subset=[
    'Clean sheets', 'Saves', 'Goals conceded', 'Errors leading to goal'
])

# Add a derived metric to rank goalkeepers
goalkeepers_cleaned['Performance Score'] = (
    (goalkeepers_cleaned['Clean sheets'] * 3) -
    (goalkeepers_cleaned['Goals conceded'] * 1) +
    (goalkeepers_cleaned['Saves'] * 2) +
    (goalkeepers_cleaned['Penalties saved'] * 3) -
    (goalkeepers_cleaned['Errors leading to goal'] * 2)
)

# Sort goalkeepers by performance score
top_goalkeepers = goalkeepers_cleaned.sort_values(by='Performance Score', ascending=False).head(20)

# Visualization
plt.figure(figsize=(12, 8))
plt.barh(top_goalkeepers['Name'], top_goalkeepers['Performance Score'], color='skyblue')
plt.xlabel('Performance Score')
plt.ylabel('Goalkeeper')
plt.title('Top 10 Goalkeepers Based on Performance Score')
plt.gca().invert_yaxis()  # Invert y-axis to have the top goalkeeper at the top
plt.show()

# Display top goalkeepers' data
top_goalkeepers[['Name', 'Club', 'Performance Score']]
